In [1]:
!wget http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv

--2022-01-23 20:18:14--  http://qim.fs.quoracdn.net/quora_duplicate_questions.tsv
Resolving qim.fs.quoracdn.net (qim.fs.quoracdn.net)... 151.101.53.2
Connecting to qim.fs.quoracdn.net (qim.fs.quoracdn.net)|151.101.53.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58176133 (55M) [text/tab-separated-values]
Saving to: ‘quora_duplicate_questions.tsv’

quora_duplicate_que 100%[===================>]  55.48M  3.05MB/s    in 24s     

2022-01-23 20:18:41 (2.35 MB/s) - ‘quora_duplicate_questions.tsv’ saved [58176133/58176133]



In [1]:
data = open('quora_duplicate_questions.tsv', 'r').readlines()
texts = []

for line in data[1:]:
    line = line.split('\t')
    try:
        text_a, text_b = line[-3], line[-2]
        if len(text_a):
            texts.append(text_a)
        if len(text_b):
            texts.append(text_b)
    except:
        pass
    
    
len(texts)

808586

In [2]:
from paddlenlp.transformers.bert.tokenizer import BasicTokenizer
from collections import Counter
from threading import Thread
from tqdm import tqdm
import json
from time import time


def saveJson(dic, path):
    with open(path, 'w') as f:
        json.dump(dic, f)
    print(path + " has been saved!")
    

def ngramGenerator(tokens, n):    
    start = 0
    end = len(tokens) - n + 1
    res = []
    for i in range(start, end):
        res.append(' '.join(tokens[i:i+n]))
    return res


def add_ngram_data(text):
    tks = tokenize(text)
    if tks:
        tks = ['<START>'] + tks + ['<END>']
        bi_tks = ngramGenerator(tks, 2)
        tri_tks = ngramGenerator(tks, 3)
        four_tks = ngramGenerator(tks, 4)
        
        unigram.update(Counter(tks))
        bigram.update(Counter(bi_tks))
        trigram.update(Counter(tri_tks))
        fourgram.update(Counter(four_tks))
        
        
def build_ngram_dataset(step=100):
    end_idx = len(texts)
    for i in tqdm(range(0, end_idx, step)):
        threads = []
        for j in range(i, i + step if i + step <= end_idx else end_idx):
            t = Thread(target=add_ngram_data, args=(texts[j], ))
            t.start()
            threads.append(t)
        for t in threads:
            t.join()


def trimCounterDic(counterDic, min_freq=0, addUNK=False):
    total = sum(counterDic.values())
    counterDic = counterDic.most_common()
    counterDic = [(c[0], c[1] / total) for c in counterDic if c[1] >= min_freq]
    if addUNK:
        counterDic.append(('<UNK>', 1 / total))
    return dict(counterDic)


def main():
    global unigram, bigram, trigram, fourgram
    
    build_ngram_dataset()
    # trim and normalize the frequency dicts
    unigram = trimCounterDic(unigram, 2, True)
    bigram = trimCounterDic(bigram, 5)
    trigram = trimCounterDic(trigram, 5)
    fourgram = trimCounterDic(fourgram, 5)
    saveJson(unigram, '../data/QuoraUnigram.json')
    saveJson(bigram, '../data/QuorBigram.json')
    saveJson(trigram, '../data/QuorTrigram.json')
    saveJson(fourgram, '../data/QuorFourgram.json')

In [3]:
if __name__ == '__main__':
    s = time()
    tokenize = BasicTokenizer().tokenize
    unigram = Counter()
    bigram = Counter()
    trigram = Counter()
    fourgram = Counter()
    main()
    e = time()
    print("Total time: " + str(e - s))

100%|███████████████████████████████████████| 8086/8086 [04:06<00:00, 32.74it/s]


../data/QuoraUnigram.json has been saved!
../data/QuorBigram.json has been saved!
../data/QuorTrigram.json has been saved!
../data/QuorFourgram.json has been saved!
Total time: 252.4117670059204
